<a href="https://colab.research.google.com/github/melbinmv/Azure_Olympic/blob/main/Azure_Olympic_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e15548199b10ac1c21707e9c53d26f9a9ffd1a3816fc453d4177584d01f53cca
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [29]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

spark = SparkSession.builder.getOrCreate()

In [15]:
#Read data using pandas
import pandas as pd
athletes_url = "https://raw.githubusercontent.com/melbinmv/Azure_Olympic/master/data/Athletes.csv"
coaches_url = "https://raw.githubusercontent.com/melbinmv/Azure_Olympic/master/data/Coaches.csv"
entriesgender_url = "https://raw.githubusercontent.com/melbinmv/Azure_Olympic/master/data/EntriesGender.csv"
medals_url = "https://raw.githubusercontent.com/melbinmv/Azure_Olympic/master/data/Medals.csv"
teams_url = "https://raw.githubusercontent.com/melbinmv/Azure_Olympic/master/data/Teams.csv"


athelets = pd.read_csv(athletes_url, encoding='latin-1')
coaches = pd.read_csv(coaches_url, encoding='latin-1')
entriesgender = pd.read_csv(entriesgender_url, encoding='latin-1')
medals = pd.read_csv(medals_url, encoding='latin-1')
teams = pd.read_csv(teams_url, encoding='latin-1')

#Convert into a spark dataframe
athletes_df = spark.createDataFrame(athelets)
coaches_df = spark.createDataFrame(coaches)
entriesgender_df = spark.createDataFrame(entriesgender)
medals_df = spark.createDataFrame(medals)
teams_df = spark.createDataFrame(teams)

In [16]:
athletes_df.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [17]:
athletes_df.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [19]:
coaches_df.show(10)

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|     NaN|
|           ABE Junya|               Japan|       Volleyball|     NaN|
|       ABE Katsuhiko|               Japan|       Basketball|     NaN|
|        ADAMA Cherif|       Côte d'Ivoire|         Football|     NaN|
|          AGEBA Yuya|               Japan|       Volleyball|     NaN|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
+--------------------+--------------------+-----------------+--------+
only s

In [20]:
coaches_df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [22]:
entriesgender_df.show(10)

+-------------------+------+----+-----+
|         Discipline|Female|Male|Total|
+-------------------+------+----+-----+
|     3x3 Basketball|    32|  32|   64|
|            Archery|    64|  64|  128|
|Artistic Gymnastics|    98|  98|  196|
|  Artistic Swimming|   105|   0|  105|
|          Athletics|   969|1072| 2041|
|          Badminton|    86|  87|  173|
|  Baseball/Softball|    90| 144|  234|
|         Basketball|   144| 144|  288|
|   Beach Volleyball|    48|  48|   96|
|             Boxing|   102| 187|  289|
+-------------------+------+----+-----+
only showing top 10 rows



In [31]:
entriesgender_df.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [30]:
entriesgender_df = entriesgender_df.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))

In [49]:
medals_df.show(10)

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
+----+--------------------+----+------+------+-----+-------------+
only showing top 10 rows



In [50]:
medals_df.printSchema()

root
 |-- Rank: long (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [41]:
medals_df = medals_df.withColumn("Gold",col("Gold").cast(IntegerType()))\
    .withColumn("Silver",col("Silver").cast(IntegerType()))\
    .withColumn("Bronze",col("Bronze").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))

In [52]:
teams_df.show(10)

+-----------+--------------+--------------------+-----+
|   TeamName|    Discipline|             Country|Event|
+-----------+--------------+--------------------+-----+
|    Belgium|3x3 Basketball|             Belgium|  Men|
|      China|3x3 Basketball|People's Republic...|  Men|
|      China|3x3 Basketball|People's Republic...|Women|
|     France|3x3 Basketball|              France|Women|
|      Italy|3x3 Basketball|               Italy|Women|
|      Japan|3x3 Basketball|               Japan|  Men|
|      Japan|3x3 Basketball|               Japan|Women|
|     Latvia|3x3 Basketball|              Latvia|  Men|
|   Mongolia|3x3 Basketball|            Mongolia|Women|
|Netherlands|3x3 Basketball|         Netherlands|  Men|
+-----------+--------------+--------------------+-----+
only showing top 10 rows



In [53]:
teams_df.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [55]:


athletes_df.select("PersonName", "Discipline").orderBy("Discipline").show()

athletes_df.select("PersonName")

+------------------+--------------+
|        PersonName|    Discipline|
+------------------+--------------+
|   BOGAERTS Rafael|3x3 Basketball|
|   PAGET Marie-Eve|3x3 Basketball|
|BAYASGALAN Solongo|3x3 Basketball|
|      YAMAMOTO Mai|3x3 Basketball|
|    BEKKERING Ross|3x3 Basketball|
|   PAWLOWSKI Pawel|3x3 Basketball|
|         BROWN Ira|3x3 Basketball|
|       KOZIK Yulia|3x3 Basketball|
|      CAVARS Agnis|3x3 Basketball|
|    PISKLOV Kirill|3x3 Basketball|
|        CELIS Nick|3x3 Basketball|
|         LI Haonan|3x3 Basketball|
|  CONSOLINI Chiara|3x3 Basketball|
|       PLUM Kelsey|3x3 Basketball|
|      CUIC Claudia|3x3 Basketball|
| ONOLBAATAR Khulan|3x3 Basketball|
|     D ALIE Raelin|3x3 Basketball|
| RATKOV Aleksandar|3x3 Basketball|
|   DOLSON Stefanie|3x3 Basketball|
|    MARIEN Thierry|3x3 Basketball|
+------------------+--------------+
only showing top 20 rows

